# Using the Qobuz API (`minim.qobuz`)

**Last updated**: June 12, 2023

In [ ]:
from copy import deepcopy
import os
import pathlib
import sys

from IPython.display import display
import ipywidgets

sys.path.insert(0, f"{pathlib.Path(os.getcwd()).parents[2].resolve()}/src")
# The second argument above should be replaced with the path where you downloaded Minim.
from minim import qobuz

## Getting started

The `minim.qobuz` submodule offers procedural and object-oriented approaches to use the Qobuz API:

* The `minim.qobuz.Session` class, which is targeted at developers and experienced programmers, implements the Qobuz API exactly on top of the `requests` library. There are no bells and whistles; responses from API calls, if any, are in JavaScript Object Notation (JSON) format, and data extraction from or wrangling of the JSON data is done solely by the user.

* The `minim.qobuz.Album`, `minim.qobuz.Artist`, `minim.qobuz.Label`, `minim.qobuz.Playlist`, `minim.qobuz.Track`, and `minim.qobuz.User` classes are more user-friendly and are meant for beginners or hobbyists. Instead of having the user make API calls directly and deal with unpacking the JSON responses, these classes do all the heavy lifting and store the information returned by the API in instance attributes that have simple, logical names and are well-documented.

Regardless of which approach above you choose, it is recommended that you have a Qobuz account and, preferably, a Qobuz streaming plan to get access to all the API endpoints.

## As a first example

To compare the procedural and object-oriented approaches and as a first introductory example, let's search for and output basic information about the track "Post Malone (feat. RANI)" by Sam Feldt.

### Procedural approach

First, we create a `qobuz.Session` object without user authentication:

In [ ]:
session_guest = qobuz.Session(authenticate=False)
print(session_guest)

As expected, the string representation of the `qobuz.Session` object confirms that no user authentication was performed. A consequence is that certain API endpoints are unavailable since they require either the user to be logged in or have an active Qobuz streaming plan.

Then, we search for tracks (or releases) with titles containing the phrase "Post Malone". For brevity, we limit our search results to only five items. The search results are in JSON format, with the first layer specifying the media type (`"albums"`, `"artists"`, `"tracks"`, etc.) and the second the number of results for that media type and Qobuz catalog information for each match. We can preview the raw data for each track by iterating:

In [ ]:
search_results = session_guest.search("Post Malone", "track", strict=True, limit=5)
for track in search_results["tracks"]["items"]:
    print(track)

The first search result is the track we are looking for. We can extract the key details of the track by traversing through the `dict` containing the Qobuz catalog information for that track:

In [ ]:
track_json_ex = search_results["tracks"]["items"][0]
print(f"Title: {track_json_ex['title']}")
print(f"Artist: {track_json_ex['performer']['name']}")
print(f"Album: {track_json_ex['album']['title']}")
print(f"Genre: {track_json_ex['album']['genre']['name']}")
print(f"Release date: {track['release_date_original']}")

In some instances, the release date information is unavailable and will be `None`.

Now, let's get the producing and songwriting credits for the track:

In [ ]:
credits = session_guest.get_track_credits(performers=track_json_ex["performers"])
for role, artists in credits.items():
    print(f"{' '.join(role.split('_')).title()}(s): {', '.join(artists)}")

What if we are building an application or widget to stream audio? We can get the URL of the track's stream using the track's Qobuz ID:

In [ ]:
print(session_guest.get_track_file_url(track_json_ex["id"])["url"])

Whoops! Because we didn't log into a Qobuz account with an active Qobuz streaming plan previously, we only got the 30-second preview of the track instead of the full audio. Let's authenticate ourselves using `qobuz.Session.login()` and try again. 

:::{note}
For the purposes of this tutorial, we will make a copy of the unauthenticated `qobuz.Session` object since we will reuse it in the upcoming sections. This is not necessary in normal use.
:::

:::{note}
No parameters are passed to `qobuz.Session.login()` below because an Qobuz user authentication token was found in my operating system's environment variable `QOBUZ_USER_AUTH_TOKEN`. If you have not stored user credentials or an authorization token in your OS's environment variables, you can pass either the `auth_token` argument or the `email` and `password` arguments to `qobuz.
:::

In [ ]:
session = deepcopy(session_guest)
session.login()
stream_url = session.get_track_file_url(track_json_ex["id"])["url"]

:::{seealso}
We can also get the track's audio stream directly as a `bytes` object. See the [Working with tracks](#working-with-tracks) section below for more information.
:::

### Object-oriented approach

First, we create a `qobuz.User` object without user authentication:

In [ ]:
user_guest = qobuz.User(authenticate=False)
print(user_guest)

As expected, the string representation of the `qobuz.User` object confirms that no user authentication was performed.

:::{note}
All objects spawned by the `qobuz.User` object shares the same Qobuz session information. Therefore, a `qobuz.Track` object created by the `qobuz.User` object above will also not be able to access information that requires user authentication. However, should the `qobuz.User` object be authenticated via `qobuz.User.login()`, the `qobuz.Track` object would also be authenticated.
:::

Like before, we search for tracks with titles containing the phrase "Post Malone", but now limit our search results to only five items. The search results are still contained in a `dict`, with the first key specifying the media type (`"albums"`, `"artists"`, `"tracks"`, etc.). However, the value corresponding to the `"tracks"` key is now a `list` of `minim.qobuz.Track` objects, each fully populated with that track's information.

In [ ]:
search_results = user_guest.search("Post Malone", "track", strict=True, limit=5)
for track in search_results["tracks"]:
    print(track)

With human-readable string representations of each track, it is easy to see that the first search result is the track we are looking for. We can extract the key details of the track by getting instance attributes of the `minim.qobuz.Track` object:

In [ ]:
track_obj_ex = search_results["tracks"][0]
print(f"Title: {track_obj_ex.title}")
print(f"Artist: {track_obj_ex.main_artist.name}")
print(f"Album: {track_obj_ex.album.title}")
print(f"Genre: {track_obj_ex.album.genre}")
print(f"Release date: {track_obj_ex.release_date}")

Now, all data in the `minim.qobuz.Track` object is human-readable, even those dealing with datetimes. Furthermore, the producing and songwriting credits for the track are already embedded in the `minim.qobuz.Track` object:

In [ ]:
for role, artists in track_obj_ex.credits.items():
    print(f"{' '.join(role.split('_')).title()}(s): {', '.join(artists)}")

Getting the audio stream data no longer requires knowledge of the track ID:

In [ ]:
track_obj_ex.get_file_url()
print(track_obj_ex.file_url)

Oops, forgot about user authentication again. This time, we authenticate using `minim.Qobuz.User.login()` instead.

:::{note}
For the purposes of this tutorial, we will make a copy of the unauthenticated `qobuz.User` object since we will reuse it in the upcoming sections. This is not necessary in normal use.
:::

In [ ]:
user = deepcopy(user_guest)
user.login()
track_obj_ex.get_file_url()

:::{seealso}
We can also get the track's audio stream directly as a `bytes` object. See the [Working with tracks](#working-with-tracks) section below for more information.
:::

## Working with artists

An artist is a person or group that performs, appears on, or contributed to an album or track. Every album, playlist, and track contains artist information in some form.

In the following section, we will retrieve information about an artist (for which we know the Qobuz ID for) using both the procedural and object-oriented approaches. For this tutorial, we will focus on the Dutch EDM duo Sick Individuals.

In [ ]:
artist_id = 764370

### Procedural approach

First, we create a `qobuz.Session` object without user authentication and use it to retrieve Qobuz catalog information for the artist. To include albums, playlists, and tracks that the artist appears on, we pass `extras=["albums", "playlists", "tracks"]` to `qobuz.Session.get_artist()`. For brevity, we limit our search results to only five items.

In [ ]:
# session_guest = qobuz.Session(authenticate=False)
# We are reusing the unauthenticated qobuz.Session object from before.

artist_json = session_guest.get_artist(artist_id, extras=["albums", "playlists", "tracks"], limit=5)
for key, value in artist_json.items():
    print(key, value, sep=": ")

We can see that the artist's albums, playlists, and tracks are stored in their respective keys above.

The only other interesting piece of information returned is a short biography of the artist:

In [ ]:
print(artist_json['biography']['content'].split('\n')[0])

### Object-oriented approach

First, we directly create a `qobuz.Artist` object using the artist's Qobuz ID. We also request albums, playlists, and tracks that the artist contributed to by including the `extras=["albums", "playlists", "tracks"]` parameter. Information about the artist is automatically retrieved.

In [ ]:
artist_obj = qobuz.Artist(artist_id, extras=["albums", "playlists", "tracks"], limit=3, authenticate=False)
print(artist_obj)

Below are the albums, playlists, and tracks that were requested:

In [ ]:
for album in artist_obj.albums:
    print(album)
for playlist in artist_obj.playlists:
    print(playlist)
for track in artist_obj.tracks:
    print(track)

Like before, a brief summary of the artist is available:

In [ ]:
print(artist_obj.biography.split("\n")[0])

:::{seealso}
While methods like `qobuz.Artist.favorite()` and `qobuz.Artist.unfavorite()` are available, it is recommended that `qobuz.User.favorite_items()` and `qobuz.User.unfavorite_items()` are used instead. See the [Account- and user-related methods](#account--and-user-related-methods) section below for more information.
:::

## Working with tracks

A track (or a release) is the most central Qobuz object. Tracks are a key part of albums and playlists and can be associated with artists who appear or feature on them.

In the following section, we will retrieve information about a track (for which we know the Qobuz ID for) using both the procedural and object-oriented approaches. For this tutorial, we will use the track "Hold On & Believe (feat. The Federal Empire)" by Martin Garrix.

In [ ]:
track_id = 35560953

### Procedural approach

First, we create a `qobuz.Session` object without user authentication and use it to retrieve Qobuz catalog information for the track:

In [ ]:
# session_guest = qobuz.Session(authenticate=False)
# We are reusing the unauthenticated qobuz.Session object from before.

track_json = session_guest.get_track(track_id)
for key, value in track_json.items():
    print(key, value, sep=": ")

The standard metadata for the track is listed below:

In [ ]:
print(f"Track: {track_json['track_number']}/{track_json['album']['tracks_count']}")
print(f"Disc: {track_json['media_number']}/{track_json['album']['media_count']}")
print(f"Title: {track_json['title']}")
print(f"Artist: {track_json['performer']['name']}")
print(f"Album: {track_json['album']['title']}")
print(f"Album artist: {track_json['album']['artist']['name']}")
print(f"Genre: {track_json['album']['genre']['name']}")
print(f"Release date: {track_json['release_date_original']}")
print(f"Copyright: {track_json['copyright']}")
print(f"ISRC: {track_json['isrc']}")

:::{note}
`track["title"]` often does not list the featured artists or contain additional information about the track (such as "KSHMR Remix", "2011 Remastered", "Taylor's Version", etc.). The featured artists can only be found in the track credits (see below), while the track version information, if any, is contained in `track["version"]`.
:::

`track["performer"]` only contains information about the primary artist associated with the track and can be incomplete. The full list of contributors can be found in `track["performers"]`:

In [ ]:
print(" -", track_json["performers"].replace(" - ", "\n - "))

The track credits above, but categorized by role, can be obtained by passing `performers=track["performers"]` (or `track_id` directly) to `qobuz.Session.get_track_credits()`:

In [ ]:
for key, value in session_guest.get_track_credits(performers=track_json["performers"]).items():
    print(key, value, sep=": ")

For streaming purposes, the file URL for the track's audio stream can be retrieved using `qobuz.Session.get_track_file_url()`. The Qobuz ID is the first (required) positional argument. The audio quality of the audio stream can optionally be specified using the `quality` argument, with the following possible values:

* `5` or `"MP3"` for constant bitrate (320 kbps) MP3
* `6` or `"CD"` for CD-quality (16-bit, 44.1 kHz) FLAC
* `7` for up to 24-bit, 96 kHz Hi-Res FLAC
* default: `27` or `"HI-RES"` for up to 24-bit, 192 kHz Hi-Res FLAC

In [ ]:
file_url = session_guest.get_track_file_url(track_id, quality="MP3")
for key, value in file_url.items():
    print(key, value, sep=": ")

For applications where the raw audio stream in `bytes` is necessary, we can use `qobuz.Session.get_track_stream()` with the same arguments instead:

In [ ]:
ipywidgets.Audio(value=session_guest.get_track_stream(track_id, quality="MP3"), autoplay=False)

### Object-oriented approach

First, we directly create a `qobuz.Track` object using the track's Qobuz ID. Information about the track is automatically retrieved.

In [ ]:
track_obj = qobuz.Track(track_id, authenticate=False)
print(track_obj)

Like before, the standard metadata for the track is listed below:

In [ ]:
print(f"Track: {track_obj.track_number}/{track_obj.album.track_count}")
print(f"Disc: {track_obj.disc_number}/{track_obj.album.disc_count}")
print(f"Title: {track_obj.title}")
print(f"Artist: {track_obj.main_artist.name}")
print(f"Album: {track_obj.album.title}")
print(f"Album artist: {track_obj.album.primary_artist.name}")
print(f"Genre: {track_obj.album.genre}")
print(f"Release date: {track_obj.release_date}")
print(f"Copyright: {track_obj.copyright}")
print(f"ISRC: {track_obj.isrc}")

Unlike before, the `qobuz.Track.title` attribute will always have information about the featured artists and track version, if available. Additional pre-processing is automatically done to ensure this, but a potential downside is that the full title is unmanageably long (like "We Are The People (feat. Bono & The Edge) [Martin Garrix Remix] [Official UEFA EURO 2020 Song]", which clocks in at *94 characters long*).

The track credits is only available pre-formatted:

In [ ]:
for key, value in track_obj.credits.items():
    print(key, value, sep=": ")

Finally, the file URL and stream data can be obtained by calling methods innate to the `qobuz.Track` object:

In [ ]:
track_obj.get_file_url(quality="MP3")
print(track_obj.file_url)

Or, if the audio stream in `bytes` is desired:

In [ ]:
track_obj.get_stream(quality="MP3")
ipywidgets.Audio(value=track_obj.stream, autoplay=False)

:::{seealso}
While methods like `qobuz.Track.favorite()` and `qobuz.Track.unfavorite()` are available, it is recommended that `qobuz.User.favorite_items()` and `qobuz.User.unfavorite_items()` are used instead. See the [Account- and user-related methods](#account--and-user-related-methods) section below for more information.
:::

## Working with albums

Albums, including extended plays (EPs) and singles, are collections of tracks by a primary artist. Every track belongs to an album.

In the following section, we will retrieve information about an album (for which we know the Qobuz ID for) using both the procedural and object-oriented approaches. For this tutorial, we will use the album 'Field Trip' by NIIKO X SWAE.

In [ ]:
album_id = "cdgt6fwyvhz3a"

### Procedural approach

First, we create a `qobuz.Session` object without user authentication and use it to retrieve Qobuz catalog information for the album:

In [ ]:
# session_guest = qobuz.Session(authenticate=False)
# We are reusing the unauthenticated qobuz.Session object from before.

album_json = session_guest.get_album(album_id)
for key, value in album_json.items():
    print(key, value, sep=": ")

Here is some basic information about the album:

In [ ]:
print(f"Title: {album_json['title']}")
print(f"Artist: {album_json['artist']['name']}")
print(f"Genre: {album_json['genre']['name']}")
print(f"Release date: {album_json['release_date_original']}")
print(f"Copyright: {album_json['copyright']}")

The JSON data also contains information about the tracks in the album:

In [ ]:
for track in album_json["tracks"]["items"]:
    print(track)

Having access to the track IDs allows for track operations, such as getting the file URLs (or audio stream data) for streaming purposes:

In [36]:
for track in album_json["tracks"]["items"]:
    print(session_guest.get_track_file_url(track["id"], quality="MP3")["url"])
    display(ipywidgets.Audio(value=session_guest.get_track_stream(track["id"], quality="MP3"), autoplay=False))

### Object-oriented approach

First, we directly create a `qobuz.Album` object using the album's Qobuz ID. 

In [ ]:
album_obj = qobuz.Album(album_id, authenticate=False)
print(album_obj)

Now, information about the album is stored in the `qobuz.Album` object's instance attributes:

In [ ]:
print(f"Title: {album_obj.title}")
print(f"Artist: {album_obj.primary_artist}")
print(f"Genre: {album_obj.genre}")
print(f"Release date: {album_obj.release_date}")
print(f"Copyright: {album_obj.copyright}")

Information about the tracks in the album are stored in `qobuz.Track` objects:

In [ ]:
for track in album_obj.tracks:
    print(track)

To get the track's file URLs or streams, we can either iterate through each `qobuz.Track` object and call its `qobuz.Track.get_file_url()` or `qobuz.Track.get_stream()` methods, or use the convenient `qobuz.Album.get_file_urls()` or `qobuz.Album.get_streams()` methods.

In [ ]:
album_obj.get_file_urls(quality="MP3")
album_obj.get_streams(quality="MP3")
for track in album_obj.tracks:
    print(track.file_url)
    display(ipywidgets.Audio(value=track.stream, autoplay=False))

:::{seealso}
While methods like `qobuz.Album.favorite()` and `qobuz.Album.unfavorite()` are available, it is recommended that `qobuz.User.favorite_items()` and `qobuz.User.unfavorite_items()` are used instead. See the [Account- and user-related methods](#account--and-user-related-methods) section below for more information.
:::

## Working with playlists

Playlists are curated collections of tracks. There are two kinds: playlists curated by Qobuz that predominantly feature tracks of the same genre or by related artists, or personal playlists created by Qobuz users.

In the following section, we will retrieve information about a public playlist (for which we know the Qobuz ID for) using both the procedural and object-oriented approaches. For this tutorial, we will use the curated playlist 'Taylor Swift' consisting of her songs.

In [ ]:
playlist_id = 2512796

### Procedural approach for public playlists

In this section, public playlists refer to playlists owned by Qobuz or other Qobuz users. Even with user authentication, you cannot edit these playlists because they do not belong to you. You can only query information about the playlists and the tracks in them.

First, we create a `qobuz.Session` object without user authentication and use it to retrieve Qobuz catalog information for the playlist:

In [ ]:
# session_guest = qobuz.Session(authenticate=False)
# We are reusing the unauthenticated qobuz.Session object from before.

playlist_json = session_guest.get_playlist(playlist_id)
for key, value in playlist_json.items():
    print(key, value, sep=": ")

Just like albums, playlists contain tracks. Here is the JSON information for the tracks:

In [ ]:
for track in playlist_json["tracks"]["items"]:
    print(track)

:::{seealso}
All track-related playlist operations are the same as those for albums. See the [Working with albums](#working-with-albums) section above for more information.
:::

### Object-oriented approach for public playlists

First, we directly create a `qobuz.Playlist` object using the playlist's Qobuz ID. 

In [ ]:
playlist_obj = qobuz.Playlist(playlist_id, authenticate=False)
print(playlist_obj)

Just like albums, information about the tracks in the playlist are stored in `qobuz.Track` objects:

In [ ]:
for track in playlist_obj.tracks:
    print(track)

:::{seealso}
All track-related playlist operations are the same as those for `qobuz.Album` objects. See the [Working with albums](#working-with-albums) section above for more information.
:::

:::{seealso}
While methods like `qobuz.Playlist.favorite()` and `qobuz.Playlist.unfavorite()` are available, it is recommended that `qobuz.User.favorite_playlist()` and `qobuz.User.unfavorite_playlist()` are used instead. See the [Account- and user-related methods](#account--and-user-related-methods) section below for more information.
:::

### Procedural approach for custom playlists

Most, if not all, custom user playlist endpoints require user authentication. As such, we now start with a `qobuz.Session` object with user authentication.

:::{note}
No parameters are passed to `qobuz.Session.login()` below because an Qobuz user authentication token was found in my operating system's environment variable `QOBUZ_USER_AUTH_TOKEN`. If you have not stored user credentials or an authorization token in your OS's environment variables, you can pass either the `auth_token` argument or the `email` and `password` arguments to `qobuz.
:::

In [ ]:
# session = qobuz.Session()

To create a playlist, we use `qobuz.Session.create_playlist()` and pass in the playlist's name as the first argument. Optionally, we can also give a description and boolean flags to determine whether the playlist is public and collaborative.

In [ ]:
user_playlist_json = session.create_playlist("Minim", description="An example custom user playlist.")
for key, value in user_playlist_json.items():
    if key != "owner":
        print(key, value, sep=": ")

In case we want to update the playlist's properties, we can use `qobuz.Session.update_playlist()`. For example, to make the playlist private, we can do:

In [ ]:
user_playlist_json = session.update_playlist(user_playlist_json["id"], public=False)
for key, value in user_playlist_json.items():
    if key != "owner":
        print(key, value, sep=": ")

Let's add some tracks to the playlist using `qobuz.Session.add_playlist_tracks()`. Afterwards, we use `qobuz.Session.get_playlist()` to query the tracks we just added.

In [ ]:
session.add_playlist_tracks(user_playlist_json["id"], 
                            [track_json_ex["id"], 
                             track_json["id"], 
                             album_json["tracks"]["items"][0]["id"],
                             playlist_json["tracks"]["items"][14]["id"]])
user_playlist_json = session.get_playlist(user_playlist_json["id"])
for track in user_playlist_json["tracks"]["items"]:
    print(track)

To reorder tracks in the playlist, use `qobuz.Session.move_playlist_tracks()`. Let's move the last track to the second position.

:::{note}
The argument `playlist_track_ids` should be the playlist track ID (which is specific to a track in a given playlist), not the Qobuz track ID.
:::

In [ ]:
session.move_playlist_tracks(user_playlist_json["id"], 
                             user_playlist_json["tracks"]["items"][2]["playlist_track_id"], 
                             0)
user_playlist_json = session.get_playlist(user_playlist_json["id"])
for track in user_playlist_json["tracks"]["items"]:
    print(track)

To delete tracks from the playlist, use `qobuz.Session.delete_playlist_tracks()`. Let's remove the third track from the playlist.

In [ ]:
session.delete_playlist_tracks(user_playlist_json["id"], 
                               user_playlist_json["tracks"]["items"][2]["playlist_track_id"])
user_playlist_json = session.get_playlist(user_playlist_json["id"])
for track in user_playlist_json["tracks"]["items"]:
    print(track)

To delete the whole playlist, use `qobuz.Session.delete_playlist()`:

In [ ]:
session.delete_playlist(user_playlist_json["id"])

### Object-oriented approach for custom playlists

To create a Qobuz playlist, we can instantiate a `qobuz.Playlist` object with the first argument being a `dict` with the arguments to pass to `qobuz.Session.create_playlist()`. To grant permissions to the object, we can pass in keyword arguments containing either an authenticated `qobuz.Session` object or user credentials. Since we already have an authenticated `qobuz.Session` object, we can do:

In [ ]:
user_playlist_obj = qobuz.Playlist({"name": "Minim"}, session=session)
user_playlist_obj.owner = {"id": None, "name": "bbye98"} # Remove personal information.
print(user_playlist_obj)

To make the playlist private, use `qobuz.Playlist.update()`:

In [ ]:
user_playlist_obj.update(public=False)
print(f"Is '{user_playlist_obj.name}' a public playlist?", user_playlist_obj.flags["public"])

Like before, let's add some tracks to the playlist using `qobuz.Playlist.add_tracks()`. Now, the tracks to be added can be provided as Qobuz track IDs or `qobuz.Track` objects. Afterwards, the `qobuz.Playlist` object is automatically updated with the tracks' information.

In [ ]:
user_playlist_obj.add_tracks([track_obj_ex.id, track_obj, album_obj.tracks[0].id, playlist_obj.tracks[14]])
for track in user_playlist_obj.tracks:
    print(track)

To reorder tracks in the playlist, use `qobuz.Playlist.move_tracks()`.

:::{note}
For a given track, normally we would have to pass its *playlist track ID*, which is a track ID specific to that track in the current playlist we are working with. However, `qobuz.Playlist.move_tracks()` also accepts a `qobuz.Track` object that did not originate from another playlist. As long as the track itself is in the playlist, the `qobuz.Track` object does not need to have a `playlist_track_id` attribute.
:::

In [ ]:
user_playlist_obj.move_tracks(album_obj.tracks[0], 0)
for track in user_playlist_obj.tracks:
    print(track)

To delete tracks from the playlist, use `qobuz.Playlist.delete_tracks()`.

In [ ]:
user_playlist_obj.delete_tracks(user_playlist_obj.tracks[2])
for track in user_playlist_obj.tracks:
    print(track)

To delete the whole playlist, use `qobuz.Playlist.delete()`. Calling this method does not delete the `qobuz.Playlist` object, but will clear all its attributes.

In [ ]:
user_playlist_obj.delete()
print(user_playlist_obj)

## Account- and user-related methods

There are a few remaining Qobuz API endpoints that do not fall into any of the categories above, mainly because they deal with account history and user preferences. Their implementations are practically one-to-one between `qobuz.Session` and `qobuz.User`, except `qobuz.User` generally allows `qobuz.Album`, `qobuz.Artist`, `qobuz.Playlist`, and `qobuz.Track` objects in-place of album IDs, artist IDs, playlist IDs, and track IDs, respectively, in the arguments for its methods. Additionally, `qobuz.User` methods store the requested information in easily accessible attributes, whereas `qobuz.Session` methods generally return data in JSON format. As such, we will use the authenticated `qobuz.User` instance from earlier in the rest of this tutorial.

### Search

Let's say we really like the color maroon. One would even say obsessively so. As such, we're interested in any media that contain the word "maroon". We can use `qobuz.User.search()` to query Qobuz for matching catalog items:

In [ ]:
results = user.search("maroon", limit=5)

The search results is returned as a `dict`, where the keys are the media type (`"albums"`, `"artists"`, or `"tracks"`) and the values are `list`s containing Qobuz catalog information. If the `qobuz.User` object is authenticated, the search results will also include the most popular albums, artists, and tracks in `"most_popular"`:

In [ ]:
for item in results["most_popular"]:
    print(item)

If we're interested only in one media type, we can specify it in the `type` parameter. There are also boolean flags we can enable to filter for high-resolution audio (`hi_res`) and new releases (`new_release`), and to force exact matches (`strict`).

For example, if we wanted to search for albums and tracks with high-resolution audio involving artists with "maroon" in their names,

In [ ]:
results_performer = user.search("maroon", type="performer", hi_res=True, limit=5)
for media_type in ("albums", "tracks"):
    print(f"{media_type.capitalize()}:")
    for item in results_performer[media_type]:
        print(f"  - {item}")
    print()

If we instead wanted to search for newly-released albums and tracks with "maroon" in the title,

In [ ]:
results_tracks = user.search("maroon", type="track", new_release=True, limit=5)
for media_type in ("albums", "tracks"):
    print(f"{media_type.capitalize()}:")
    for item in results_tracks[media_type]:
        print(f"  - {item}")
    print()

### Favorites

To add albums, tracks, and artists to your favorites, use `qobuz.User.favorite_items()`:

In [ ]:
user.favorite_items(albums=album_json["id"], artists=artist_json["id"], tracks=track_json_ex["id"])
for key, value in user.favorites.items():
    print(f"Favorite {key}:")
    for item in value:
        print(f"  - {item}")
    print()

For playlists, use `qobuz.User.favorite_playlist()`:

In [ ]:
user.favorite_playlist(playlist_json["id"])
for playlist in user.playlists:
    print(playlist)

To unfavorite albums, tracks, or artists, use `qobuz.User.unfavorite_items()`. For playlists, use `qobuz.User.unfavorite_playlist()`.

In [ ]:
user.unfavorite_items(albums=album_json["id"], artists=artist_json["id"], tracks=track_json_ex["id"])       
user.unfavorite_playlist(playlist_json["id"])

### Curated and featured media

Every week, Qobuz prepares curated tracks and highlights featured albums and playlists depending on the user's listening history.

To get the curated tracks, use `qobuz.User.get_curated_tracks()`:

In [ ]:
for track in user.get_curated_tracks(limit=5):
    print(track)

To get the featured albums, use `qobuz.User.get_featured_albums()`:

In [ ]:
for album in user.get_featured_albums(limit=5):
    print(album)

And lastly, to get the featured playlists, use `qobuz.User.get_featured_playlists()`:

In [ ]:
for playlist in user.get_featured_playlists(limit=5):
    print(playlist)